https://medium.com/swlh/automated-weather-model-processing-with-foss4g-lessons-learned-8aaaeda1e3bc

In [1]:
import io
from datetime import datetime, timedelta, timezone

import numpy as np
import psycopg
from psycopg import sql
import xarray as xr
from numpy import typing as npt
import pandas as pd
from typing import Any

In [2]:
pg_connection_dict = {
    'dbname': "mydb",
    'user': "myn",
    'password': r"IE>AUs]6)fr>jQ8m",
    'port': "5432",
    'host': "terraform-20221218184631507400000002.c2x7llrlmsr3.us-east-2.rds.amazonaws.com"
}

with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
    with conn.cursor() as curr:
        print(conn.info.encoding)
        print(curr.execute(r"SELECT now()").fetchall())

utf-8
[(datetime.datetime(2022, 12, 21, 19, 25, 13, 691312, tzinfo=datetime.timezone.utc),)]


In [12]:
with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
    with conn.cursor() as curr:
        curr.execute(open("../database.sql", "rt").read())

In [65]:
ds = xr.open_dataset(
    "../data/CMC_reg_SNOD_SFC_0_ps10km_2022121518_P001.grib2",
    engine="cfgrib",
    indexpath="",
)

In [66]:
def to_utcdatetime(date: np.datetime64):
    """
    Converts a numpy datetime64[ns] object to a python datetime object
    Input:
      date - a np.datetime64[ns] object
    Output:
      DATE - a python datetime object
    """
    timestamp = int(
        (date - np.datetime64("1970-01-01T00:00:00")) / np.timedelta64(1, "s")
    )
    return datetime.fromtimestamp(timestamp, timezone.utc)


def to_timedelta(td: np.timedelta64):
    """
    Converts a numpy timedelta64[ns] object to a python timedelta object
    Input:
      date - a np.datetime64[ns] object
    Output:
      DATE - a python datetime object
    """
    return timedelta(td.astype("timedelta64[h]").astype("float") / 24)

In [67]:
ds_var = list(ds.keys())[0]

variable = {
        "short_name": ds[ds_var].attrs["GRIB_shortName"],
        "long_name": ds[ds_var].attrs["GRIB_name"],
        "unit": ds[ds_var].attrs["units"],
    }

In [68]:
with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
    with conn.cursor() as curr:
        curr.execute("""
        INSERT INTO public.variables(short_name, long_name, unit) VALUES (
            %(short_name)s,
            %(long_name)s,
            %(unit)s
        );""", variable)

In [69]:
forecast = {
    "model": "rdps",
    "reference_time": to_utcdatetime(ds["time"].values),
    "step": to_timedelta(ds["step"].values),
}

with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
    with conn.cursor() as curr:
        curr.execute("""
        INSERT INTO public.forecasts(model, forecast_reference_time, forecast_step) VALUES (
            %(model)s,
            %(reference_time)s,
            %(step)s
        );""", forecast)

In [70]:
columns = ["forecast_id", "variable_id", "longitude", "latitude", "value"]

write_query = sql.SQL("INSERT INTO {table} ({columns}) VALUES ({placeholders})").format(
    table=sql.Identifier("public", "predictions"),
    columns=sql.SQL(', ').join(map(sql.Identifier, columns)),
    placeholders=sql.SQL(', ').join(sql.Placeholder() * len(columns))
)

In [71]:
def write_data():
    with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
        with conn.cursor() as curr:
            with conn.transaction():
                forecast_id = curr.execute(
                    """
                    SELECT forecast_id
                    FROM public.forecasts
                    WHERE 
                        model = %(model)s AND
                        forecast_reference_time = %(reference_time)s AND
                        forecast_step = %(step)s
                """,
                    forecast,
                ).fetchone()
            if not forecast_id:
                raise ValueError("No forecast_id obtained")

            with conn.transaction():
                variable_id = curr.execute(
                    """
                    SELECT variable_id
                    FROM public.variables
                    WHERE short_name = %s
                """,
                    [variable["short_name"]],
                ).fetchone()
            
            if not variable_id:
                raise ValueError("No variable_id obtained")

            # Prepare data to upload
            df = ds.to_dataframe().fillna({ds_var: 0})
            df = df[["longitude", "latitude", ds_var]].reset_index(drop=True)
            df = df.loc[:, ["longitude", "latitude", ds_var]]
            df = df.rename(columns={ds_var: "value"})
            df["forecast_id"] = forecast_id[0]
            df["variable_id"] = variable_id[0]
            df = df.loc[
                :, ["forecast_id", "variable_id", "longitude", "latitude", "value"]
            ]

            curr.executemany(write_query, list(df.itertuples(index=False)))

In [72]:
write_data()

In [13]:
# https://weather.gc.ca/grib/grib2_HRDPS_HR_e.html
# https://weather.gc.ca/grib/HRDPS_HR/lamnatpoints.gz

def write_coordinates():
    coords_file = "../data/coordinates/hrdps_continential.txt"
    df = pd.read_csv(coords_file, sep=" ", names=["i", "j", "latitude", "longitude"], skiprows=1)

    columns = ["latitude", "longitude"]
    write_query = sql.SQL("COPY {table} ({columns}) FROM STDIN").format(
        table=sql.Identifier("public", "coordinates"),
        columns=sql.SQL(', ').join(map(sql.Identifier, columns)),
    )

    with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
            with conn.cursor() as curr:
                with curr.copy(write_query) as copy:
                    for record in df[["latitude", "longitude"]].values:
                        copy.write_row(record)
write_coordinates()

In [8]:
coords_file = "../data/coordinates/hrdps_continential.txt"
df = pd.read_csv(coords_file, sep=" ", names=["i", "j", "latitude", "longitude"], skiprows=1)
df.head()

,i,j,latitude,longitude
0,1,1,35.603376,-128.082536
1,1,2,35.609918,-128.060513
2,1,3,35.616454,-128.038485
3,1,4,35.622983,-128.016450
4,1,5,35.629506,-127.994409


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750656 entries, 0 to 3750655
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   i          int64  
 1   j          int64  
 2   latitude   float64
 3   longitude  float64
dtypes: float64(2), int64(2)
memory usage: 114.5 MB


In [21]:
def execute_sql_as_dataframe(sql_query: sql.Composed) -> pd.DataFrame:
    with psycopg.connect(**pg_connection_dict, autocommit=True) as conn:
            with conn.cursor() as curr:
                res = curr.execute(sql_query).fetchall()
                if curr.description:
                    columns = [col.name for col in curr.description]
                    return pd.DataFrame(res, columns=columns)
                return pd.DataFrame()

In [22]:
query = sql.SQL("SELECT {fields} FROM {table}").format(
    fields=sql.SQL(", ").join(map(sql.Identifier, ["coord_id", "latitude", "longitude"])),
    table=sql.Identifier("public","coordinates")
)

res = execute_sql_as_dataframe(query)

In [26]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750655 entries, 0 to 3750654
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   coord_id   int64  
 1   latitude   float64
 2   longitude  float64
dtypes: float64(2), int64(1)
memory usage: 85.8 MB
